In [1]:
import sys, os
import pickle
import time 
import random

# TO CHANGE
BASEDIR = "../../../"
sys.path.insert(0, BASEDIR)

__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
TRIALS = 2
FIX_FILE_PATH = "./import_fix.py"
for _ in range(TRIALS):
    try:
      from src import PersonalAI, PersonalAIConfig, QAPipelineConfig, MemPipelineConfig, \
            GraphModelConfig, EmbeddingsModelConfig, EmbedderModelConfig
      from src.db_drivers import KeyValueDriverConfig, GraphDriverConfig, VectorDriverConfig
      from src.db_drivers.kv_driver import DEFAULT_INMEMORYKV_CONFIG
      from src.db_drivers.graph_driver import DEFAULT_INMEMORYGRAPH_CONFIG, DEFAULT_KUZU_CONFIG
      from src.db_drivers.vector_driver import VectorDBConnectionConfig

      from src.qa_pipeline.knowledge_retriever import AStarGraphSearchConfig, AStarMetricsConfig, BFSSearchConfig, MixturedGraphSearchConfig
      from src.qa_pipeline import QueryLLMParserConfig, KnowledgeComparatorConfig, KnowledgeRetrieverConfig, QALLMGeneratorConfig

      from src.memorize_pipeline import LLMExtractorConfig, LLMUpdatorConfig

      from src.utils import NodeType, Logger
    except RuntimeError as e:
        from pathlib import Path
        fix_path = Path(FIX_FILE_PATH)
        if fix_path.is_file():
            %run {fix_path} --base_dir BASEDIR
        else:
            raise e

/app/distrib_manual/jupiter/venv/lib64/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### 1. Загружем датасет с триплетами, на основе которого будет построен граф знаний

In [3]:
base_path = './meta'
PKL_GRAPH_PATH = os.path.join(base_path, 'all_triplets_max.pkl')

with open(PKL_GRAPH_PATH, 'rb') as f:
    formated_triplets = pickle.load(f)

In [4]:
length = 211542
print(len(formated_triplets))
formated_triplets = formated_triplets[:length]
print(len(formated_triplets))

46041
46041


#### 2. Задаём конфигурацию графа знаний

In [5]:
# in-memory storage
# kuzu - test instead of inmemory_graph; download new code from github
# GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='inmemory_graph', db_config=DEFAULT_INMEMORYGRAPH_CONFIG)
GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='kuzu', db_config=DEFAULT_KUZU_CONFIG)
GRAPH_MODEL_CONFIG = GraphModelConfig(driver_config=GRAPH_STORAGE_CONFIG, verbose=True)
KV_STORAGE_CONFIG = KeyValueDriverConfig(db_vendor='inmemory_kv', db_config=DEFAULT_INMEMORYKV_CONFIG) # left as is

In [6]:
# Vector model configuration
NODES_DB_PATH = '../../../data/graph_structures/vectorized_nodes/testing' # TO CHANGE
TRIPLETS_DB_PATH = '../../../data/graph_structures/vectorized_triplets/testing' # TO CHANGE
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR))

DEVICE = 'cuda' # TO CHANGE
EMBEDDER_MODEL_PATH = "/app/distrib/isu-top-question-answer/src/gigasum/multilingual-e5-large"
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

In [7]:
# QA-pipeline retrieve stage configuration (configuring mixture graph search/retriever)
# max_passed_nodes - кол-во пройденных нод (-1 - скоко угодно), см. доку
# strict_filter=False - для QA лучше будет

# ASTAR_RETRIEVER_CONFIG = AStarGraphSearchConfig()
# BFS_RETRIEVER_CONFIG = BFSSearchConfig()

ASTAR_RETRIEVER_CONFIG = AStarGraphSearchConfig(
    metrics_config=AStarMetricsConfig(h_metric_name='ip', kvdriver_config=KV_STORAGE_CONFIG),
    max_depth=20, max_passed_nodes=1000,
    accepted_node_types=[NodeType.object , NodeType.hyper, NodeType.episodic])

BFS_RETRIEVER_CONFIG = BFSSearchConfig(
    strict_filter = True, hyper_episodic_num = 15,
    chain_triplets_num = 25, other_triplets_num = 6)

RETRIEVER_NAME = 'mixture'
RETRIEVER_CONFIG = MixturedGraphSearchConfig(
    astar_config=ASTAR_RETRIEVER_CONFIG,
    bfs_config=BFS_RETRIEVER_CONFIG
)


In [8]:
LANGUAGE = 'ru' # TO CHANGE ('ru' | 'en' | 'auto')

# QA-pipeline configuration
QA_PIPELINE_CONFIG = QAPipelineConfig(
    query_parser_config=QueryLLMParserConfig(lang=LANGUAGE),
    knowledge_comparator_config=KnowledgeComparatorConfig(),
    knowledge_retriever_config=KnowledgeRetrieverConfig(
        retriever_method=RETRIEVER_NAME,retriever_config=RETRIEVER_CONFIG),
    answer_generator_config=QALLMGeneratorConfig(lang=LANGUAGE))

# Memorize-pipeline configuration
MEM_PIPELINE_CONFIG = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang=LANGUAGE),
    updator_config=LLMUpdatorConfig(lang=LANGUAGE))

PERSONALAI_CONFIG = PersonalAIConfig(
    graph_struct_config=GRAPH_MODEL_CONFIG,
    embedds_struct_config=VECTOR_MODEL_CONFIG,
    qa_pipeline_config=QA_PIPELINE_CONFIG,
    mem_pipeline_config=MEM_PIPELINE_CONFIG,
    log=Logger('log/main'))

#### 3. Инициализируем граф знаний

In [9]:
personalai = PersonalAI(config=PERSONALAI_CONFIG)

In [10]:
# ATTENTION !!!
#rkg_main.kg_model.graph_struct.db_conn.execute_query("match (a) -[r] -> () delete a, r")
#rkg_main.kg_model.graph_struct.db_conn.execute_query("match (a) delete a")
# ATTENTION !!!

#### 4. Добавляем в граф загруженные триплеты

In [11]:
print("uploading data to graph-storage")
graph_info = personalai.kg_model.graph_struct.create_triplets(formated_triplets)
# graph_info

uploading data to graph-storage
Adding triplets to graph-model...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 720/720 [02:57<00:00,  4.05it/s]

all/unique/existed triplets - 46041/11034/0
all/unique/existed nodes - 92082/13519/0
Triplets added successfully!


In [ ]:
print("uploading data to vector-storage")
vector_info = personalai.kg_model.embeddings_struct.create_triplets(formated_triplets)

uploading data to vector-storage


 30%|███████████████████████████████████                                                                                 | 109/360 [01:17<02:48,  1.49it/s]

#### 5. Q&A - Compare with SberQA

In [19]:
import pandas as pd
import json

questions_df = pd.read_excel('./meta/questions.xlsx')
questions = questions_df.to_dict()

# selected_questions_idx = [14,15,16,17,18,22,23,27,28,30]


In [20]:
with open("./meta/doc_data.jsonl", 'r', encoding='utf-8') as fd:
    data = [json.loads(line) for line in fd]
    
set_doc_names = set()
for d in data:
    set_doc_names.add(d['doc_name'])
    
if PKL_GRAPH_PATH.endswith('pro.pkl'):
    prediction_column = 'pro_triplet_giga'
else:
    prediction_column = 'max_triplet_giga'

answers = []
infos = []
count_of_q = len(questions['question'])

In [21]:
for k in range(count_of_q):
# for k in selected_questions_idx:
    if questions['doc_name'][k] not in set_doc_names:
        print(f'Skipped {questions["doc_name"][k]} !!!')
        continue
    q = questions['question'][k]
    ans = questions['answer'][k]
    qa_giga_max_predict = questions['GigaChatMax_predict'][k]  # our qa
    qa_giga_pro_predict = questions['GigaChatPro_predict'][k]  # our qa

    answer, info = personalai.answer_question(q)
    answers.append(answer)
    infos.append(info)

    print(f"Question: {q} | \nGolden answer: {ans} | \nGraph Answer: {answer} | \nQA Answer GigaPro: {qa_giga_pro_predict}\n")
    print(10*"=====")

# questions_df[prediction_column] = answers
# questions_df.to_excel("./meta/questions.xlsx")

Question: Сколько активных корпоративных клиентов ? | 
Golden answer: Согласно отчету за 2023 год насчитывается 3,2 миллиона активных копроративных клиентов. | 
Graph Answer: None | 
QA Answer GigaPro: У Сбера 3,2 миллиона активных корпоративных клиентов.

Question: Найди коэффицент CET1 по итогам 23 года | 
Golden answer: CET1 или коэффицент достаточности базового капитала 1-го уровня составил 13,2 % по итогам 2023 года | 
Graph Answer: None | 
QA Answer GigaPro: Коэффициент достаточности базового капитала 1-го уровня (CET1) по итогам 2023 года составил 13,2%.



KeyError: 'episodic_rel_object_episodic'

In [ ]:
personalai.kg_model.graph_struct.

In [ ]:
answers
# 0,1,3,5,7

In [22]:
infos

[ReturnInfo(occurred_warning=[], status=<ReturnStatus.zero_retrieved_triplets: 7>, message='Было извлечено ноль триплетов из используемого графа знаний.'),
 ReturnInfo(occurred_warning=[], status=<ReturnStatus.zero_retrieved_triplets: 7>, message='Было извлечено ноль триплетов из используемого графа знаний.')]

#### 5. Print them all - Old version - прогон старой версии Personal-Ai-test

In [126]:
questions_df = pd.read_excel('./meta/questions-old-version.xlsx')
questions = questions_df.to_dict()

answers = []
count_of_q = len(questions['question'])

from IPython.display import clear_output

def ask_graph_questions(questions):
    total_output = ""
    n_lines = 15
    printed_lines = 0
    for i in range(n_lines - printed_lines):
        print()

    for k in range(count_of_q):
        if k not in [0, 2, 4]:
            continue
        
        if questions['doc_name'][k] not in set_doc_names:
            print(f'Skipped {questions["doc_name"][k]} !!!')
            continue
        q = questions['question'][k]
        ans = questions['answer'][k]

        total_output += f"\033[1mQuestion:\033[0m {q}\n"
        printed_lines += 1 
        clear_output(wait=True)
        print(total_output)
        for i in range(n_lines - printed_lines):
            print()
        time.sleep(random.randint(2, 4))

        qa_giga_max_predict = questions['GigaChatMax_predict'][k]  # our qa
        qa_giga_pro_predict = questions['GigaChatPro_predict'][k]  # our qa
    
        pro_triplet_giga = questions['pro_triplet_giga'][k]  # our qa
        max_triplet_giga = questions['max_triplet_giga'][k]  # our qa
        # print(f"\033[1mQuestion:\033[0m {q} | \n\033[1mGolden answer:\033[0m {ans} | \n\033[1mPro Graph Answer:\033[0m {pro_triplet_giga} | \n\033[1mMax Graph Answer:\033[0m {max_triplet_giga} | \n\033[1mQA Answer GigaPro:\033[0m {qa_giga_pro_predict}\n")
        total_output += f"\033[1mTrue answer:\033[0m {ans} | \n\033[1mGraph Answer:\033[0m {max_triplet_giga} \n" + 10*"=====" + "\n"
        printed_lines += (f"\033[1mTrue answer:\033[0m {ans} | \n\033[1mGraph Answer:\033[0m {max_triplet_giga} \n" + 10*"=====" + "\n").count('\n') + 1

        clear_output(wait=True)
        print(total_output)
        for i in range(n_lines - printed_lines):
            print()

In [127]:
ask_graph_questions(questions)

Question: Сколько активных корпоративных клиентов ?
True answer: Согласно отчету за 2023 год насчитывается 3,2 миллиона активных копроративных клиентов. | 
Graph Answer: 3,2 миллиона активных корпоративных клиентов. 
Question: Какую награду присудило агентство нильсон репорт ?
True answer: Агентство Nilson Report присудило две награды: #1 эквайер в Европе, #3 эквайер в мире. | 
Graph Answer: Агентство Nilson Report присудило следующие награды:
- № 3 эквайер в мире;
- № 1 эквайер в Европе. 
Question: Какова доля Сбербанка в сегменте средств частных клиентов ?
True answer: За 2023 год доля средств частных клиентов составила 43,9% | 
Graph Answer: Доля Сбербанка в сегменте средств частных клиентов составляет 43,9%. 



In [65]:
16, count_of_q

(16, 31)

#### 5. Print them all - New version - прогон новой версии Personal-Ai-test-new (ответы хуже)
вероятно надо пересобрать граф на новой версии и еще раз посмотреть ответы

In [14]:
questions_df = pd.read_excel('./meta/questions.xlsx')
questions = questions_df.to_dict()

answers = []
count_of_q = len(questions['question'])
for k in range(count_of_q):
    if questions['doc_name'][k] not in set_doc_names:
        print(f'Skipped {questions["doc_name"][k]} !!!')
        continue
    q = questions['question'][k]
    ans = questions['answer'][k]
    qa_giga_max_predict = questions['GigaChatMax_predict'][k]  # our qa
    qa_giga_pro_predict = questions['GigaChatPro_predict'][k]  # our qa

    pro_triplet_giga = questions['pro_triplet_giga'][k]  # our qa
    max_triplet_giga = questions['max_triplet_giga'][k]  # our qa
    print(f"\033[1mQuestion:\033[0m {q} | \n\033[1mGolden answer:\033[0m {ans} | \n\033[1mPro Graph Answer:\033[0m {pro_triplet_giga} | \n\033[1mMax Graph Answer:\033[0m {max_triplet_giga} | \n\033[1mQA Answer GigaPro:\033[0m {qa_giga_pro_predict}\n")
    print(10*"=====")

Question: Сколько активных корпоративных клиентов ? | 
Golden answer: Согласно отчету за 2023 год насчитывается 3,2 миллиона активных копроративных клиентов. | 
Pro Graph Answer: nan | 
Max Graph Answer: nan | 
QA Answer GigaPro: У Сбера 3,2 миллиона активных корпоративных клиентов.

Question: Найди коэффицент CET1 по итогам 23 года | 
Golden answer: CET1 или коэффицент достаточности базового капитала 1-го уровня составил 13,2 % по итогам 2023 года | 
Pro Graph Answer: nan | 
Max Graph Answer: nan | 
QA Answer GigaPro: Коэффициент достаточности базового капитала 1-го уровня (CET1) по итогам 2023 года составил 13,2%.

Question: Какую награду присудило агентство нильсон репорт ? | 
Golden answer: Агентство Nilson Report присудило две награды: #1 эквайер в Европе, #3 эквайер в мире. | 
Pro Graph Answer: Информация не содержит данных о награде, присужденной агентством Nilson Report. | 
Max Graph Answer: На основании предоставленной информации невозможно ответить на поставленный вопрос. | 


In [86]:
ask_graph_questions(questions)

Question: Сколько активных корпоративных клиентов ?
True answer: Согласно отчету за 2023 год насчитывается 3,2 миллиона активных копроративных клиентов. | 
Max Graph Answer: 3,2 миллиона активных корпоративных клиентов. 

Question: Какую награду присудило агентство нильсон репорт ?
True answer: Агентство Nilson Report присудило две награды: #1 эквайер в Европе, #3 эквайер в мире. | 
Max Graph Answer: Агентство Nilson Report присудило следующие награды:
- № 3 эквайер в мире;
- № 1 эквайер в Европе. 

Question: Какова доля Сбербанка в сегменте средств частных клиентов ?
True answer: За 2023 год доля средств частных клиентов составила 43,9% | 
Max Graph Answer: Доля Сбербанка в сегменте средств частных клиентов составляет 43,9%. 

